In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [3]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("BEARER_TOKEN")

def create_url(id):
    tweet_fields = "tweet.fields=lang"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    ids = "ids=" + id
    #
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    return url

In [4]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer <BearerToken>"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r

In [5]:
def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [6]:
def get_tweet(id):
    url = create_url(id)
    json_response = connect_to_endpoint(url)
    return json_response

In [7]:
df_early = pd.read_fwf('Covid_Tweets_03.03.20-10.03.20.txt', header = None)
df_late = pd.read_fwf('Covid_Tweets_27.11.20-03.12.20.txt', header = None)

In [56]:
def reset():
    #Early
    early_ids = df_early.iloc[:,0].values.tolist()
    
    early_id_list = []
    while len(early_ids) > 100 :
        early_id_str = ""
        for i in range(100):
            if(i<99):
                early_id_str = early_id_str+str(early_ids.pop())+","
            else:
                early_id_str = early_id_str+str(early_ids.pop())
        early_id_list.append(early_id_str)
    early_id_str = ""
    x = len(early_ids)
    for i in range(x):
            if(i<x-1):
                early_id_str = early_id_str+str(early_ids.pop())+","
            else:
                early_id_str = early_id_str+str(early_ids.pop())
    early_id_list.append(early_id_str)

    del early_id_list[5000:]
    
    
    #Late
    late_ids = df_late.iloc[:,0].values.tolist()
    
    late_id_list = []
    while len(late_ids) > 100 :
        late_id_str = ""
        for i in range(100):
            if(i<99):
                late_id_str = late_id_str+str(late_ids.pop())+","
            else:
                late_id_str = late_id_str+str(late_ids.pop())
        late_id_list.append(late_id_str)
    late_id_str = ""
    x = len(late_ids)
    for i in range(x):
            if(i<x-1):
                late_id_str = late_id_str+str(late_ids.pop())+","
            else:
                late_id_str = late_id_str+str(late_ids.pop())
    late_id_list.append(late_id_str)
    print(len(late_id_list))
    late_id_list = del late_id_list[5000:]

In [48]:
def get_early_tweets():  
    file = open('early-covid-tweets.csv', 'w+', newline ='', encoding='utf-8') 
    with file: 
        while len(early_id_list) > 300:
            for i in range(300):
                json_tweet = get_tweet(early_id_list.pop())
                if "data" in json_tweet.keys():
                    for x in json_tweet['data']:
                        if x['lang'] == "en":
                            write = csv.writer(file)
                            y = str(x['text'])
                            write.writerow([y]) 
            print("sleep")
            time.sleep(900)
        x = len(early_id_list)
        for i in range(x):
            json_tweet = get_tweet(early_id_list.pop())
            if "data" in json_tweet.keys():
                    for x in json_tweet['data']:
                        if x['lang'] == "en":
                            write = csv.writer(file) 
                            y = str(x['text'])
                            write.writerow([y]) 

In [52]:
def get_late_tweets():  
    file = open('late-covid-tweets.csv', 'w+', newline ='', encoding='utf-8') 
    with file: 
        while len(late_id_list) > 300:
            print(len(late_id_list))
            for i in range(300):
                json_tweet = get_tweet(late_id_list.pop())
                if "data" in json_tweet.keys():
                    for x in json_tweet['data']:
                        if x['lang'] == "en":
                            write = csv.writer(file)
                            y = str(x['text'])
                            write.writerow([y]) 
            time.sleep(900)
        x = len(late_id_list)
        for i in range(x):
            json_tweet = get_tweet(late_id_list.pop())
            if "data" in json_tweet.keys():
                    for x in json_tweet['data']:
                        if x['lang'] == "en":
                            write = csv.writer(file) 
                            y = str(x['text'])
                            write.writerow([y]) 

In [65]:
reset()
get_early_tweets()
get_late_tweets()

5320
3473
3173
2873
2573
2273
1973
1673
1373
1073
773
473
